아래 aidev의 코드를 이용하였음을 알려드립니다 
<br>

http://aidev.co.kr/chatbotdeeplearning/8709
<br>


# **준비 사항**

In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

<br>
<br>

# **데이터 로드**

In [ ]:
# 네이버 영화리뷰 감정분석 데이터 다운로드
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


박은정님의 네이버 영화리뷰 감정분석 데이터를 Github에서 다운로드 합니다.
<br>
nsmc 디렉토리에 raw data와 labeling 된 데이터들이 존재합니다. 
<br>
해당 데이터를 라벨링하는데 사용한 코드는 nsmc/code에서 확인할 수 있습니다. 해당 코드를 참조해 좀 더 엄격한 labeling 기준을 적용한 데이터셋을 만들어 실험을 진행했습니다.
<br>

In [ ]:
# 디렉토리의 파일 목록
!ls nsmc -la

total 38644
drwxr-xr-x 5 root root     4096 Jun 29 09:43 .
drwxr-xr-x 1 root root     4096 Jun 29 11:42 ..
drwxr-xr-x 2 root root     4096 Jun 29 09:43 code
drwxr-xr-x 8 root root     4096 Jun 29 09:43 .git
-rw-r--r-- 1 root root  4893335 Jun 29 09:43 ratings_test.txt
-rw-r--r-- 1 root root 14628807 Jun 29 09:43 ratings_train.txt
-rw-r--r-- 1 root root 19515078 Jun 29 09:43 ratings.txt
drwxr-xr-x 2 root root   466944 Jun 29 09:43 raw
-rw-r--r-- 1 root root     2596 Jun 29 09:43 README.md
-rw-r--r-- 1 root root    36746 Jun 29 09:43 synopses.json


nsmc의 raw 데이터를 가공해서 새롭게 labeling 된 데이터를 만듭니다

In [ ]:
from collections import defaultdict
from glob import glob
import os
import re
import time

from lxml import html
import numpy as np
import pandas as pd
import requests

sub_space = lambda s: re.sub('\s+', ' ', s)
write_row = lambda l, f: f.write('\t'.join(l) + '\n')

num_data = 0

filenames = glob('nsmc/raw/*')
with open('ratings_all.txt', 'w') as f:
    write_row('id document label'.split(), f)
    for filename in filenames:
        review = pd.read_json(filename)
        for n in range(len(review)):
          rating = review['rating'][n]
          if rating > 9:      # positive 10
              try:
                  f.write('%s\t%s\t%s\n' % (review['review_id'][n], sub_space(review['review'][n]), '1'))
                  num_data += 1
              except:
                  pass
          elif rating < 3:    # negative 1 2
              try:
                  f.write('%s\t%s\t%s\n' % (review['review_id'][n], sub_space(review['review'][n]), '0'))
                  num_data += 1
              except:
                  pass
          else:               # neutral
              pass
          if num_data == 200000:
            break
        if num_data == 200000:
          break
num_data

200000

150000개의 train set과
<br>
50000개의 test set을 골라냅니다

In [ ]:
import numpy as np; np.random.seed(1234)
import pandas as pd

ntrain = 150000
ntest = 50000

data = pd.read_csv('ratings_all.txt', sep='\t', quoting=3)
data = pd.DataFrame(np.random.permutation(data))
trn, tst = data[:ntrain], data[ntrain:ntrain+50000]

header = 'id document label'.split()
trn.to_csv('new_ratings_train.txt', sep='\t', index=False, header=header)
tst.to_csv('new_ratings_test.txt', sep='\t', index=False, header=header)

In [ ]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("new_ratings_train.txt", sep='\t')
test = pd.read_csv("new_ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


훈련셋 150,000개와 테스트셋 50,000개의 데이터가 존재합니다.
<br>

In [ ]:
# 훈련셋의 앞부분 출력
train.head(10)

,id,document,label
0,4483118,원덜포,1
1,2886015,어렸을 때 굉장히 감명깊게 보고 아직도 잊혀지지가 않아요,1
2,10101330,"이 할아방구 노망나신듯.... 이게 섬나라의 인간들의 한계인듯,,,",0
3,9931433,말이 필요하냐...?,1
4,2116879,윌리암 스콧과 더 락의 만남. 적절한 캐스팅과 적절한 무난함이다,0
5,10128496,10년만에 다시 보니 새롭군요.정의에 대해 많은 생각을 하게 해 주어서 좋습니다.,1
6,7411200,이건아냐 정말..열심히 그렸으니 그래도 2점 줌..최악의애니,0
7,5340081,단순하고 평범해보이기까지 해서 더 묵직한 정복자의 어린 시절.,0
8,6053034,"뜻밖에 건진 드라마 한 편, 너무 짧아 아쉽긴 했지만 강해창의 사랑에 마음이 짠했다",1
9,8798317,추억의 영화니까 10점주마,1


id는 회원정보, document는 리뷰 문장입니다. label이 0이면 부정, 1이면 긍정으로 분류됩니다. id는 사용하지 않기 때문에 document와 label만 추출하겠습니다. 

<br>
<br>

# **전처리 - 훈련셋**

In [ ]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

0                                               원덜포
1                   어렸을 때 굉장히 감명깊게 보고 아직도 잊혀지지가 않아요
2             이 할아방구 노망나신듯.... 이게 섬나라의 인간들의 한계인듯,,,
3                                       말이 필요하냐...?
4               윌리암 스콧과 더 락의 만남. 적절한 캐스팅과 적절한 무난함이다
5     10년만에 다시 보니 새롭군요.정의에 대해 많은 생각을 하게 해 주어서 좋습니다.
6                 이건아냐 정말..열심히 그렸으니 그래도 2점 줌..최악의애니
7                단순하고 평범해보이기까지 해서 더 묵직한 정복자의 어린 시절.
8    뜻밖에 건진 드라마 한 편, 너무 짧아 아쉽긴 했지만 강해창의 사랑에 마음이 짠했다
9                                    추억의 영화니까 10점주마
Name: document, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 원덜포 [SEP]',
 '[CLS] 어렸을 때 굉장히 감명깊게 보고 아직도 잊혀지지가 않아요 [SEP]',
 '[CLS] 이 할아방구 노망나신듯.... 이게 섬나라의 인간들의 한계인듯,,, [SEP]',
 '[CLS] 말이 필요하냐...? [SEP]',
 '[CLS] 윌리암 스콧과 더 락의 만남. 적절한 캐스팅과 적절한 무난함이다 [SEP]',
 '[CLS] 10년만에 다시 보니 새롭군요.정의에 대해 많은 생각을 하게 해 주어서 좋습니다. [SEP]',
 '[CLS] 이건아냐 정말..열심히 그렸으니 그래도 2점 줌..최악의애니 [SEP]',
 '[CLS] 단순하고 평범해보이기까지 해서 더 묵직한 정복자의 어린 시절. [SEP]',
 '[CLS] 뜻밖에 건진 드라마 한 편, 너무 짧아 아쉽긴 했지만 강해창의 사랑에 마음이 짠했다 [SEP]',
 '[CLS] 추억의 영화니까 10점주마 [SEP]']

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

BERT의 입력은 위의 그림과 같은 형식입니다. Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입됩니다. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 합니다. [SEP]은 Seperation을 가리키는데, 두 문장를 구분하는 역할을 합니다. 이 예제에서는 문장이 하나이므로 [SEP]도 하나만 넣습니다.
<br>
<br>
<br>

In [ ]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 1, 0, ..., 1, 1, 1])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 원덜포 [SEP]
['[CLS]', '원', '##덜', '##포', '[SEP]']


BERT는 형태소분석으로 토큰을 분리하지 않습니다. WordPiece라는 통계적인 방식을 사용합니다. 한 단어내에서 자주 나오는 글자들을 붙여서 하나의 토큰으로 만듭니다. 이렇게 하면 언어에 상관없이 토큰을 생성할 수 있다는 장점이 있습니다. 또한 신조어 같이 사전에 없는 단어를 처리하기도 좋습니다. 

위의 결과에서 ## 기호는 앞 토큰과 이어진다는 표시입니다. 토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용합니다. 그래서 한글도 처리가 가능합니다.
<br>
<br>
<br>

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9612, 118783,  55530,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

보통 딥러닝 모델에는 토큰 자체를 입력으로 넣을 수 없습니다. 임베딩 레이어에는 토큰을 숫자로 된 인덱스로 변환하여 사용합니다. BERT의 토크나이저는 {단어토큰:인덱스}로 구성된 단어사전을 가지고 있습니다. 이를 참조하여 토큰을 인덱스로 바꿔줍니다.
<br>
<br>
<br>

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   8965,  10622,   8962,  11018, 118817,  11102,   9995,  14871,
         15387,   9657,  14867,  10459, 100208,    119,    102,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
# 배치 사이즈
batch_size = 64

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

<br>
<br>

# **전처리 - 테스트셋**

In [ ]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                         진짜 재미있다. 나랑은 좀 안맞지만 뭐 이정도면..
1          산자의 고통, 죽은자의 그리움을 그 경계면에서 한바탕 굿으로 풀어낸 느낌...
2    보면서 이상하다 했더니 리뷰에 있네요.. 일본우익애들이 만든것 같은 만화.. 성우들...
3                음, HSBC은행과 Cairo far east bank만큼의 차이?
4                      앞으로 술마실때 혹시 메탄올 아닌지 잘 보고 마시도록.ㅋ
5                           진짜 너무 잼있었어요.. 서변의 포스도 짱짱..
6            성동일이 죽는 부분까지는 괜찮았는데., 그 이후는 그저 그런 영화로....
7                                  감정을 턱끝까지 차오르게 했던 영화
8                                          정우성이 빛나는 영화
9                            슈퍼맨이 영 크리스토퍼 삘이 안나서 별루...
Name: document, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 진짜 재미있다. 나랑은 좀 안맞지만 뭐 이정도면.. [SEP]',
 '[CLS] 산자의 고통, 죽은자의 그리움을 그 경계면에서 한바탕 굿으로 풀어낸 느낌... [SEP]',
 '[CLS] 보면서 이상하다 했더니 리뷰에 있네요.. 일본우익애들이 만든것 같은 만화.. 성우들 손발 오그라 들어서 힘드셨을거 같네요.. [SEP]',
 '[CLS] 음, HSBC은행과 Cairo far east bank만큼의 차이? [SEP]',
 '[CLS] 앞으로 술마실때 혹시 메탄올 아닌지 잘 보고 마시도록.ㅋ [SEP]',
 '[CLS] 진짜 너무 잼있었어요.. 서변의 포스도 짱짱.. [SEP]',
 '[CLS] 성동일이 죽는 부분까지는 괜찮았는데., 그 이후는 그저 그런 영화로.... [SEP]',
 '[CLS] 감정을 턱끝까지 차오르게 했던 영화 [SEP]',
 '[CLS] 정우성이 빛나는 영화 [SEP]',
 '[CLS] 슈퍼맨이 영 크리스토퍼 삘이 안나서 별루... [SEP]']

In [ ]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 진짜 재미있다. 나랑은 좀 안맞지만 뭐 이정도면.. [SEP]
['[CLS]', '진', '##짜', '재', '##미', '##있다', '.', '나', '##랑', '##은', '좀', '안', '##맞', '##지만', '뭐', '이', '##정', '##도', '##면', '.', '.', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9708, 119235,   9659,  22458,  76820,    119,   8982,
        62200,  10892,   9682,   9521, 118913,  28578,   9303,   9638,
        16605,  12092,  14867,    119,    119,    102,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9708, 119235,   9659,  22458,  76820,    119,   8982,  62200,
         10892,   9682,   9521, 118913,  28578,   9303,   9638,  16605,  12092,
         14867,    119,    119,    102,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
# 배치 사이즈
# batch_size = 512

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

<br>
<br>

# **모델 생성**

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

![대체 텍스트](http://www.mccormickml.com/assets/BERT/padding_and_mask.png)

사전훈련된 BERT는 다양한 문제로 전이학습이 가능합니다. 여기서는 위의 그림과 같이 한 문장을 분류하는 방법을 사용합니다. 영화리뷰 문장이 입력으로 들어가면, 긍정/부정으로 구분합니다. 모델의 출력에서 [CLS] 위치인 첫 번째 토큰에 새로운 레이어를 붙여서 파인튜닝을 합니다. Huggning Face는 BertForSequenceClassification() 함수를 제공하기 때문에 쉽게 구현할 수 있습니다.
<br>
<br>
<br>

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 1

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

<br>
<br>

# **모델 학습**

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [70]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...
  Batch   500  of  2,110.    Elapsed: 0:20:16.


KeyboardInterrupt: ignored

에폭마다 훈련셋과 검증셋을 반복하여 학습을 수행합니다. 

<br>
<br>

# **테스트셋 평가**

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

<br>
<br>

# **새로운 문장 테스트**

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits = test_sentences(['연기는 별로지만 재미 하나는 끝내줌!'])

print(logits)
print(np.argmax(logits))

In [ ]:
logits = test_sentences(['주연배우가 아깝다. 총체적 난국...'])

print(logits)
print(np.argmax(logits))

학습한 모델을 가지고 실제 문장을 넣어봤습니다. 출력 로짓은 소프트맥스가 적용되지 않은 상태입니다. argmax로 더 높은 값의 위치를 라벨로 설정하면 됩니다. 0은 부정, 1은 긍정입니다. 위와 같이 새로운 문장에도 잘 분류를 하고 있습니다.
<br>
<br>
<br>

다운받은 데이터를 넣고 돌려봅니다.

In [ ]:
import pandas as pd
ko = pd.read_csv("./ko_data.csv")
ko_data = [['Id', 'Predicted']]

for idx in range(len(ko['Sentence'])):
  sen = ko['Sentence'][idx]
  logit = test_sentences([sen])
  ko_data.append([idx, np.argmax(logit)])

dataframe = pd.DataFrame(ko_data)
dataframe.to_csv("test.csv", header=False, index=False)

< 챗봇 개발자 모임 ><br>
- 페이스북 그룹에 가입하시면 챗봇에 대한 최신 정보를 쉽게 받으실 수 있습니다.
- https://www.facebook.com/groups/ChatbotDevKR/